In [7]:
import glymur
import numpy as np 
import matplotlib.pyplot as plt
import os
import skimage
import pickle as pkl
from sklearn.decomposition import PCA
from skimage.measure import compare_ssim as ssim
import random

%run -i 'dataset_models.py'
%run -i 'new_utilities.py'
%run -i 'train_nn.py'
%run -i 'nn_models.py'
%run -i 'collect_data.py'

Collecting data first. There are helper functions for CIFAR-10, MNIST, Faces, FSDD datasets. For a custom dataset, each row must be a data point.

In [ ]:
data_dir = '' #Specify data directory
train_data, train_labels, test_data, test_labels = collect_data_cifar10(data_dir)

To run accuracy experiments, we divide the training data into two parts. One to train for the compression algorithm and the other to train for the classification task.

In [ ]:
train_data_alg, train_data_acc, train_labels_acc = split_train_test(train_data, train_labels, split=0.6)
test_data_alg = np.concatenate((train_data_acc,test_data))

Create a dataset object and metrics objects for each algorithm. You can either pass the entire dataset and specify the fraction of training data, or pass the train and test dataset separately. 

In [ ]:
cifar_10_dataset = Dataset(np.array([train_data_alg,test_data_alg]))
metrics_pca = Metrics(accuracy=1)
metrics_pba = Metrics(accuracy=1)
metrics_jpeg = Metrics(accuracy=1)
metrics_jpeg2k = Metrics(accuracy=1)

Following cells contain code to obtain rate-distortion points for PCA, PBA, JPEG, JPEG2000 where distortion metrics are SNR, SSIM or classification accuracy.

In [ ]:
#PCA
num_comp_l = np.concatenate((np.array([3,5]),
                             (np.logspace(1,np.log10(cifar_10_dataset.num_tr/2),num=5, endpoint=False)).astype(np.uint16),
                             (np.logspace(np.log10(cifar_10_dataset.num_tr/2),np.log10(cifar_10_dataset.num_tr),num=10)).astype(np.uint16))) 
for num_components in num_comp_l:
    cifar_10_dataset = quant_pca(cifar_10_dataset,num_components,a=900)
    metrics_pca = compute_snr_ssim_metrics(cifar_10_dataset,metrics_pca,algorithm='pca',multichannel=True)
    metrics_pca = cifar_compute_accuracy(cifar_10_dataset,train_labels_acc,test_labels,metrics_pca,'pca',epochs=2)

In [ ]:
#PBA
a=900
lmb_vec = np.logspace(np.log2(np.min(cifar_10_dataset.pca_eigval/(8*(a-1)))),np.log2(np.max(cifar_10_dataset.pca_eigval/(4*(a-1))) - 1),base=2,num=30)
for lmb in lmb_vec:
    cifar_10_dataset = quant_pba(cifar_10_dataset,lmb,a)
    metrics_pba = compute_snr_ssim_metrics(cifar_10_dataset,metrics_pba,algorithm='pba',multichannel=True)
    metrics_pba = cifar_compute_accuracy(cifar_10_dataset,train_labels_acc,test_labels,metrics_jpeg,'pba',epochs=2)     

In [ ]:
#JPEG
for qual in range(1,101,4):
    cifar_10_dataset,metrics_jpeg = quant_jpegfamily(cifar_10_dataset,metrics_jpeg,qual,multichannel=True)
    metrics_jpeg = cifar_compute_accuracy(cifar_10_dataset,train_labels_acc,test_labels,metrics_pba,'jpeg',epochs=2) 
#JPEG2000    
cratio = [1,2,3,5,8,10,13,15,20,30,50,75,100,1000]
for ratio in cratio:
    cifar_10_dataset,metrics_jpeg2k = quant_jpegfamily(cifar_10_dataset,metrics_jpeg2k,ratio,multichannel=True)
    metrics_jpeg2k = cifar_compute_accuracy(cifar_10_dataset,train_labels_acc,test_labels,metrics_jpeg2k,'jpeg2k',epochs=2) 